In [3]:
def initiate_path():
    path_temp = 'output'
    path_round_about_temp = f'{path_temp}/roundabout'
    path_network_temp = f'{path_temp}/networks'
    return path_temp, path_round_about_temp, path_network_temp

In [4]:
import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io

path, path_round_about, path_network = initiate_path()
project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Polygon, Point, LineString, MultiPolygon, MultiPoint
import math
import warnings

warnings.filterwarnings(action='ignore')

### Run only when download for first time

In [2]:
# Create graph from OSM server and project it
graph = ox.graph_from_place('Torino', network_type='all')
graph = ox.bearing.add_edge_bearings(graph, precision=1)
graph_pro = ox.projection.project_graph(graph, to_crs=project_crs)
io.save_graph_geopackage(graph_pro, filepath='.', encoding='utf-8', directed=False)

KeyboardInterrupt: 

### End

In [4]:
# delete segments without names
my_gdf = gpd.read_file(f'{path}/edges.shp')
not_null = my_gdf.dropna(subset='name')
not_null.to_file(f'{path}/not_null.shp')
# delete segments present any kind of roundabout or tunnel
not_roundaout = not_null[~((not_null['junction'] == 'roundabout') | (not_null['junction'] == 'circular') | (
            not_null['tunnel'] == 'building_passage') | (not_null['tunnel'] == 'yes') | (
                                       not_null['highway'] == 'cycleway') | (not_null['highway'] == 'path'))]
round_about = not_null[not_null['junction'].isin(['roundabout', 'circular'])]
not_roundaout.to_file(f'{path}/not_roundabout_tunnel.shp')
round_about.to_file(f'{path_round_about}/roundabout.shp')
# Project dataframe and calculate angle (0 to 180)
df_pro = not_roundaout.to_crs(project_crs)
df_pro['angle'] = df_pro['bearing'].apply(lambda x: x if x < 180 else x - 180)
df_pro['length'] = df_pro.length
df_pro.to_file(f'{path}/pro.shp')

In [4]:
df_pro = gpd.read_file(f'{path}/pro.shp')

In [5]:
def length_of_parallel(my_s_join: GeoDataFrame, the_buffer: GeoSeries, geo_field: str) -> int:
    my_s_join['geometry'] = my_s_join[geo_field]
    new_data_0 = my_s_join.sjoin(GeoDataFrame(geometry=the_buffer, crs=project_crs), how='inner').reset_index()
    if len(new_data_0) == 0:
        return 0
    return len(new_data_0[new_data_0['index'] != new_data_0['index_right']])

In [6]:
def check_parallelism(to_translate: GeoDataFrame, is_test_local: bool = False) -> bool:
    my_buffer = to_translate['geometry'].buffer(cap_style=2, distance=30, join_style=3)
    to_translate['geometry_right'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'right'))
    to_translate['geometry_left'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'left'))
    # Currently is it not working
    if is_test_local:
        # to_translate.drop(columns= ['geometry', new_geometry[0]]).rename(columns = {new_geometry[1]: 'geometry'}).to_file(f'{path}/test_data/res_translate_{new_geometry[1]}.shp')
        # to_translate.drop(columns= ['geometry', new_geometry[1]]).rename(columns = {new_geometry[0]: 'geometry'}).to_file(f'{path}/test_data/res_translate_{new_geometry[0]}.shp')
        my_buffer.to_file(f'{path}/test_data/buffers_test.shp')
    if length_of_parallel(to_translate, my_buffer, 'geometry_right') > 0 or length_of_parallel(to_translate, my_buffer,
                                                                                               'geometry_left') > 0:
        return True
    else:
        return False

In [40]:
def simplify(my_polygon: Polygon, x: (int, float)) -> Polygon:
    simplify_poly = my_polygon.simplify(x, preserve_topology=False)
    if simplify_poly.area < 50:
        simplify_poly = simplify(my_polygon, x / 2)
    return simplify_poly

In [45]:
def create_center_line(one_poly):
    """
    This method calculate new line between the farthest points of the simplified polygon
    :param one_poly:
    :return:
    """

    pnt_list = one_poly.exterior.coords[:-1]
    list_shp = [Point(item) for item in pnt_list]
    dis, dis_2, dis_3 = 0, 0, 0
    third_dis = (-1, -1)
    # The new line will be determined by the second-farthest points
    for k, point in enumerate(list_shp):
        j = k
        for point2 in list_shp[k + 1:]:
            j += 1
            temp_dis = point.distance(point2)
            if temp_dis > dis:
                dis = point.distance(point2)
            elif temp_dis > dis_2:
                dis_2 = point.distance(point2)
            elif temp_dis > dis_3:
                third_dis = (k, j)
                dis_3 = point.distance(point2)
    if is_test:
        dic_sim = {'index': [0], 'geometry': one_poly}
        GeoDataFrame(dic_sim, crs='epsg:3857').to_file(f'{path}/test_data/simplify_poly_{unit[0]}_{id_pol + 2}.shp')
    max_dist['name'].extend([id_pol + 2, id_pol + 2])
    max_dist['geometry'].extend([list_shp[third_dis[0]], list_shp[third_dis[1]]])

In [12]:
def update_df_with_center_line(new_line):
    """
    update our dictionary with new lines
    :param new_line:
    :return:
    """
    dic_final['name'].append(name)
    # dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]], pnt_list[max_dis[1]])))
    dic_final['geometry'].append(new_line)
    dic_final['highway'].append(data.iloc[0]['highway'])
    dic_final['bearing'].append(data['angle'].mean())
    dic_final['group'].append(unit[0])


In [13]:
def update_list(line_local):
    """
    add the first start/end point into the list
    :param line_local:
    :return:
    """
    list_pnts_of_line_group.extend([Point(line_local.coords[0]), Point(line_local.coords[-1])])


In [14]:
def add_more_pnts_to_new_lines(pnt_f_loc: Point, pnt_l_loc: Point, line_pnts: list) -> list:
    """
    This method checks if more points should be added to the new lines by checking along the new line if the distance to the old network roads are more than 10 meters
    :return:
    """
    # Calculate distance and azimuth between the first and last point
    dist = pnt_f_loc.distance(pnt_l_loc)
    x_0 = pnt_f_loc.coords[0][0]
    y_0 = pnt_f_loc.coords[0][1]
    bearing = math.atan2(pnt_l_loc.coords[0][0] - x_0, pnt_l_loc.coords[0][1] - y_0)
    bearing = bearing + 2 * math.pi if bearing < 0 else bearing
    # Calculate the number of  checks going to carry out
    loops = int(dist / 100)

    # Calculate  the first point over the line
    for dis_on_line in range(1, loops):
        x_new = x_0 + 100 * dis_on_line * math.sin(bearing)
        y_new = y_0 + 100 * dis_on_line * math.cos(bearing)
        # S_joins to all the network lines (same name and group)
        # if the distance is less than 10 meters continue, else: find the projection point and add it to the correct location and run the function agein
        one_pnt_df = GeoDataFrame(geometry=[Point(x_new, y_new)], crs=project_crs)
        s_join_loc = one_pnt_df.sjoin_nearest(data, distance_col='dis').iloc[0]
        if s_join_loc['dis'] > 10:
            pnt_med = s_join_loc['geometry']
            line = data.loc[s_join_loc['index_right']]['geometry']
            line_pnts.append(line.interpolate(line.project(pnt_med)))
            line_pnts = add_more_pnts_to_new_lines(pnt_med, pnt_l_loc, line_pnts)
            return line_pnts
    return line_pnts

In [46]:
# Main point to start

is_test = False
my_groupby = df_pro.groupby('name')
dic_final = {'name': [], 'geometry': [], 'highway': [], 'bearing': [], 'group': []}

for_time = len(my_groupby)
for i, street in enumerate(my_groupby):
    # Calculate time to run
    print(f'{round(i / for_time * 100, 2)}\t', end="")
    res = street[1]
    name = street[0]
    if is_test:
        name = 'Corso Giovanni Agnelli'
        res = my_groupby.get_group(name)
    # groupby angle
    res = res.dropna(subset=['angle'], axis=0)
    if len(res) == 0:
        continue
    res['group'] = DBSCAN(eps=5, min_samples=2).fit(res['angle'].to_numpy().reshape(-1, 1)).labels_
    cur_group = res[res['group'] > -1].groupby('group')
    is_parallel = False
    for group in cur_group:
        data = group[1]
        if check_parallelism(data):
            # if among of lines with same angles some are parallel,find the center line for each group
            is_parallel = True
            for unit in cur_group:
                data = unit[1]

                # new points DataFrame of start/end line of each group
                list_pnts_of_line_group = []
                data['geometry'].apply(update_list)
                df_pnts = GeoDataFrame(geometry=list_pnts_of_line_group, crs='epsg:3857').drop_duplicates()

                # unify lines to one polygon
                buffers = data.buffer(cap_style=3, distance=30, join_style=3)
                one_buffer = buffers.unary_union

                max_dist = {'name': [], 'geometry': []}
                # simplify polygon with simplify function. If one_buffer is multipolygon object simplify each one them separately
                if isinstance(one_buffer, MultiPolygon):
                    for id_pol, polygon in enumerate(one_buffer):
                        create_center_line(polygon)
                else:
                    id_pol = -1
                    create_center_line(one_buffer)
                max_df = GeoDataFrame(max_dist, crs='epsg:3857')
                # find for each points the closet point from the oribinal data. the closet points will create the new line
                s_join = max_df.sjoin_nearest(df_pnts).groupby('name')
                for geo in s_join:
                    same_name = geo[1]
                    if same_name.iloc[0]['index_right'] == same_name.iloc[1]['index_right']:
                        continue
                    in_0 = same_name.iloc[0]['index_right']
                    in_1 = same_name.iloc[1]['index_right']
                    # These points will be served to be initial reference in order to find more points
                    pnt_f = df_pnts.loc[in_0]['geometry']
                    pnt_l = df_pnts.loc[in_1]['geometry']
                    lines_pnt_geo = add_more_pnts_to_new_lines(pnt_f, pnt_l, [pnt_f])
                    lines_pnt_geo.append(pnt_l)
                    # Updata dic_final
                    update_df_with_center_line(LineString(lines_pnt_geo))
                    # dic_final['geometry'].append(LineString(lines_pnt_geo))
                    # new_lines['name'].append(geo[0])
                if is_test:
                    buffers.to_file(f'{path}/test_data/buffers_{unit[0]}.shp')
                    # dic_one = {'index':[0],'geometry':one_buffer.geoms}
                    # GeoDataFrame(dic_one,crs='epsg:3857').to_file(f'{path}/test_data/one_buffer_{unit[0]}.shp')
        # if one group is found as parallel all the groups are calculated as parallel so we don't need to check_parallelism
        if is_parallel:
            break
    # # טסט
    if is_test:
        final = GeoDataFrame(dic_final, crs=project_crs)
        final = final[final.length > 20]
        final.to_file(f'{path}/test_data/one_line.shp')
        res.to_file(f'{path}/test_data/groups.shp')
        break

if not is_test:
    print('create new files')
    # remove short lines
    final_cols = ['name', 'geometry', 'highway', 'bearing', 'length']
    final = GeoDataFrame(dic_final, crs=project_crs)
    final['is_simplify'] = 1
    final['length'] = final.length
    final = final[final.length > 100]
    final.to_file(f'{path}/one_line_third.shp')
    # create network
    new_network = df_pro[~df_pro['name'].isin(dic_final['name'])][final_cols]
    new_network['is_simplify'] = 0
    new_network.append(final).to_file(f'{path_network}/new_network_third.shp')

0.0	0.04	0.08	0.12	0.16	0.2	0.24	0.28	0.32	0.37	0.41	0.45	0.49	0.53	0.57	0.61	0.65	0.69	0.73	0.77	0.81	0.85	0.89	0.93	0.97	1.01	1.06	1.1	1.14	1.18	1.22	1.26	1.3	1.34	1.38	1.42	1.46	1.5	1.54	1.58	1.62	1.66	1.7	1.75	1.79	1.83	1.87	1.91	1.95	1.99	2.03	2.07	2.11	2.15	2.19	2.23	2.27	2.31	2.35	2.39	2.44	2.48	2.52	2.56	2.6	2.64	2.68	2.72	2.76	2.8	2.84	2.88	2.92	2.96	3.0	3.04	3.08	3.12	3.17	3.21	3.25	3.29	3.33	3.37	3.41	3.45	3.49	3.53	3.57	3.61	3.65	3.69	3.73	3.77	3.81	3.86	3.9	3.94	3.98	4.02	4.06	4.1	4.14	4.18	4.22	4.26	4.3	4.34	4.38	4.42	4.46	4.5	4.55	4.59	4.63	4.67	4.71	4.75	4.79	4.83	4.87	4.91	4.95	4.99	5.03	5.07	5.11	5.15	5.19	5.24	5.28	5.32	5.36	5.4	5.44	5.48	5.52	5.56	5.6	5.64	5.68	5.72	5.76	5.8	5.84	5.88	5.93	5.97	6.01	6.05	6.09	6.13	6.17	6.21	6.25	6.29	6.33	6.37	6.41	6.45	6.49	6.53	6.57	6.62	6.66	6.7	6.74	6.78	6.82	6.86	6.9	6.94	6.98	7.02	7.06	7.1	7.14	7.18	7.22	7.26	7.31	7.35	7.39	7.43	7.47	7.51	7.55	7.59	7.63	7.67	7.71	7.75	7.79	7.83	7.87	7.91	7.95	8.0	8.04	8.08	8.12	8.16	8.2	8.24	8

1. Delete false intersection
2. Split in intersection
3. Remove short
4. Roundabout
5. Extend
6. Split in intersection
7. Remove short
8. Update roundabout
9. consolidate intersections


In [6]:
# Fix topology
from momepy import remove_false_nodes,extend_lines
cur_path = f'{path}/rearrange'

Delete false intersection

In [39]:
new_gpd = gpd.read_file(f'{path_network}/new_network_third.shp')

# First clean all the false node
remove_false_f= remove_false_nodes(new_gpd)
# the previous function has changed the topology so the length should be updated
remove_false_f['length'] =remove_false_f.length
remove_false_f.to_file(f'{cur_path}/remove_false_nodes.shp')

Split in intersection

In [61]:
def find_intersection_points(row):
    r"""
    find the intersection points between the two lines
    :param row:
    :return:
    """
    try:
        line_1 = remove_false_f.loc[row['FID']]
        line_2 =  remove_false_f.loc[row['index_right']]
        pnt = line_1.geometry.intersection(line_2.geometry)
        # If there are more than one intersection between two lines, one of the lines should be deleted.
        if isinstance(pnt,MultiPoint):
            temp_line= line_1.name if line_1.length< line_2.length else line_2.name
            if temp_line not in lines_to_delete:
                lines_to_delete.append(temp_line)
            return
        # If it is first or end continue OR if there is no intersection between the two lines
        if len(pnt.coords)==0 or pnt.coords[0]==line_1.geometry.coords[0] or pnt.coords[0]==line_1.geometry.coords[-1]:
            return
        inter_pnt_dic['geometry'].append(pnt)
        inter_pnt_dic['name'].append(row['FID'])
    except:
        print(f"{row['FID']},{row['index_right']}:{pnt}")


In [24]:
# Create buffer around each element
buffer_around_lines= remove_false_f['geometry'].buffer(cap_style=3, distance=1, join_style=3)
buffer_around_lines.to_file(f'{cur_path}/buffer.shp')

# s_join between buffer to lines
s_join_0 =gpd.sjoin(left_df=buffer_around_lines,right_df=remove_false_f)

# delete lines belong to the buffer
s_join = s_join_0[s_join_0['FID']!=s_join_0['index_right']]
s_join.to_file(f'{cur_path}/s_join.shp')

# Find new intersections that are not at the beginning or end of the line
lines_to_delete =[]
inter_pnt_dic = {'geometry':[],'name':[]}
s_join.apply(find_intersection_points, axis=1)
inter_pnt_gdf = GeoDataFrame(inter_pnt_dic,crs=project_crs)
inter_pnt_gdf.to_file(f'{cur_path}/inter_pnt.shp')



In [ ]:
# Split string line by points
segments = {'geometry':[],'org_id':[]}
# Groupby points name (which is the line they should split)
for group_pnts in inter_pnt_gdf.groupby('name'):
    points  = group_pnts[1]
    points['is_split'] = True

    # get the line to split by comparing the name
    row = remove_false_f.loc[group_pnts[0]]
    current = list(row.geometry.coords)
    points_line = [Point(x) for x in current]
    points_line_gdf = GeoDataFrame(geometry=points_line,crs=project_crs)
    points_line_gdf['is_split'] = False

    # append all the points together (line points and split points)
    line_all_pnts = points_line_gdf.append(points)

    # Find the distance of each point form the begining of the line on the line.
    line_all_pnts['dis_from_the_start'] = line_all_pnts['geometry'].apply(lambda x:row.geometry.project(x))
    line_all_pnts.sort_values('dis_from_the_start',inplace=True)

    # split the line
    seg =[]
    for point in line_all_pnts.iterrows():
        prop = point[1]
        seg.append(prop['geometry'])
        if prop['is_split']:
            segments['geometry'].append(LineString(seg))
            segments['org_id'].append(row.name)
            seg = [prop['geometry']]
    segments['geometry'].append(LineString(seg))
    segments['org_id'].append(row.name)
network_split = GeoDataFrame(data=segments,crs=project_crs)
cols_no_geometry = remove_false_f.columns[:-1]
network_split_final = network_split.set_index('org_id')
network_split_final[cols_no_geometry] =remove_false_f[cols_no_geometry]
network_split_final.to_file(f'{cur_path}/only_split.shp')
# remove old and redundant line from our network and update with new one
network_split = remove_false_f.drop(index=network_split_final.index.unique()).append(network_split_final).drop(index= lines_to_delete)

In [220]:
network_split['length'] = network_split.length
network_split.to_file(f'{cur_path}/split.shp')

In [20]:
# This is not part of the process. It allows us to start at the middle
buffer_around_lines  =gpd.read_file(f'{cur_path}/buffer.shp')
remove_false_f =gpd.read_file(f'{cur_path}/remove_false_nodes.shp')
s_join =gpd.read_file(f'{cur_path}/s_join.shp')
s_join.rename(columns = {'index_righ':'index_right'}, inplace = True)
network_split =  gpd.read_file(f'{cur_path}/split.shp')

Roundabout

In [21]:
# Find the center of each roundabout
# create polygon around each polygon and union
round_about = gpd.read_file(f'{path_round_about}/roundabout.shp')
round_about_buffer = round_about.to_crs(project_crs)['geometry'].buffer(cap_style=1, distance=10,
                                                                        join_style=1).unary_union
dic_data = {'name': [], 'geometry': []}
for ii, xx in enumerate(round_about_buffer):
    dic_data['name'].append(ii)
    dic_data['geometry'].append(xx.centroid)
GeoDataFrame(dic_data, crs=project_crs).to_file(f'{path_round_about}/centroid.shp')
# GeoDataFrame(dic_data,crs=project_crs).to_file(f'{path_round_about}/roundabout_union.shp')

In [18]:
def populate_pnt_dic(point: type, name_of_line: str):
    """
    Make "pnt_dic" contain a list of all the lines connected to each point.
    :param point:
    :param name_of_line:
    :return:
    """
    if not point in pnt_dic:
        pnt_dic[point] = []
    pnt_dic[point].append(name_of_line)

In [16]:
def send_pnts(temp_line: GeoSeries):
    """
    # Send the first and the last points to populate_pnt_dic
    :return:
    """
    my_geom = temp_line['geometry']
    populate_pnt_dic(my_geom.coords[0], temp_line.name)
    populate_pnt_dic(my_geom.coords[-1], temp_line.name)

In [24]:
def first_last_pnt_of_line(row: GeoSeries):
    r"""
    It get geometry of line and fill the first_last_dic with the first and last point and the name of the line
    :return:
    """
    geo = list(row['geometry'].coords)
    first_last_dic['geometry'].extend([Point(geo[0]), Point(geo[-1])])
    first_last_dic['line_name'].extend([row.name] * 2)
    first_last_dic['position'].extend([0, -1])

In [25]:
def deadend(path_to_save: str, network: GeoDataFrame, export_files: bool = False):
    r"""
    remove not connected line shorter than 100 meters and then return deadend_list lines and their endpoints (as another file)
    :param network:
    :param path_to_save:
    :param export_files:
    :return:
    """

    network.apply(send_pnts, axis=1)
    deadend_list = [item[0] for item in pnt_dic.values() if len(item) == 1]

    # save all the deadend_list points
    deadend_pnts_list = [item[0] for item in pnt_dic.items() if len(item[1]) == 1]

    # Create two files one with all not connected lines and second after removing shorter then 200 meters
    not_connected_lines = [j for p, j in enumerate(deadend_list) if deadend_list[:p].count(j) == 1]
    not_connected_gdf = network.loc[not_connected_lines]
    delete_short = not_connected_gdf[not_connected_gdf.length > 100]

    # Get deadend_list and remove short unconnected
    deadend_list = [j for v, j in enumerate(deadend_list) if deadend_list[:v].count(j) == 0]
    deadend_gdf = network.loc[deadend_list]
    short = not_connected_gdf[not_connected_gdf.length < 100]
    drop_short = deadend_gdf.drop(index=short.index)

    # Create gdf of line points with the reference to the line they belong
    drop_short.apply(first_last_pnt_of_line, axis=1)
    first_last_gdf = GeoDataFrame(first_last_dic, crs=project_crs)

    # Make sure the points are dead end
    point_as_tuple = 'point_as_tuple'
    first_last_gdf[point_as_tuple] = first_last_gdf.geometry.apply(lambda x: x.coords[0])
    first_last_gdf = first_last_gdf[first_last_gdf[point_as_tuple].isin(deadend_pnts_list)].drop(point_as_tuple, axis=1)

    if export_files:
        # Optional - Create new files
        first_last_gdf.to_file(f'{path_to_save}/first_last_pnts.shp')
        not_connected_gdf.to_file(f'{path_to_save}/not_connected.shp')
        delete_short.to_file(f'{path_to_save}/not_connected_delete_short.shp')
        deadend_gdf.to_file(f'{path_to_save}/deadend_gdf_all.shp')
        drop_short.reset_index().to_file(f'{path_to_save}/deadend_lines.shp')
    return drop_short, first_last_gdf

In [26]:
def update_geometry(cur):
    r"""
    :return:
    """
    if cur['highway'] == 'footway':
        # Don't snap footway to roundabout
        return cur['geometry']
    points_lines = s_join[s_join['line_name'] == cur.name]
    if len(points_lines) == 0:
        # No roundabout nearby
        return cur['geometry']
    # get the line geometry to change the first and/ or last point
    geo_cur = list(cur['geometry'].coords)

    if len(points_lines) == 1:
        # One line point near roundabout
        geo_cur[points_lines['position'].item()] = centroid.loc[points_lines['index_right'].item()]['geometry'].coords[
            0]
    if len(points_lines) == 2:
        # teo line point near roundabout
        for ind in range(2):
            points_line = points_lines.iloc[ind]
            geo_cur[points_line['position'].item()] =centroid.loc[points_line['index_right'].item()]['geometry'].coords[0]
    return LineString(geo_cur)

In [ ]:
# Network Improvement - to roundabout
# Find unconnected lines
path_deadend = f'{path}/deadend'
new_gpd = gpd.read_file(f'{path_network}/split.shp')

In [27]:
new_gpd.rename(columns={'name': 'str_name'}, inplace=True)
pnt_dic = {}
first_last_dic = {'geometry': [], 'line_name': [], 'position': []}
deadend_lines, deadend_pnts = deadend(path_to_save=path_deadend, network=new_gpd)

# Spatial join between roundabout centroid to nearby dead end lines
centroid = gpd.read_file(f'{path_round_about}/centroid.shp')
s_join = gpd.sjoin_nearest(left_df=deadend_pnts, right_df=centroid, how='left', max_distance=60,
                           distance_col='dist').dropna(subset='dist')
s_join.to_file(f'{path_deadend}/roundabout_near_pnts.shp')
# Update the geometry so the roundabout will be part of the line geometry
change_geo = deadend_lines.copy()

change_geo['geometry'] = change_geo.apply(update_geometry, axis=1)
change_geo.reset_index().to_file(f'{path_deadend}/connect_roundabout.shp')

# update the current network
new_network_temp = new_gpd.drop(index=change_geo.index)
new_network2 = new_network_temp.append(change_geo)
new_network2.to_file(f'{path_network}/network_ra.shp')

Remove line short then 20 meters

Roundabout

## things to improve the code - Roundabout :
1. Snap all lines near roundabout to the roundabout
2. Replace the nearest points on the line with the roundabout (and not the end points)
3. Till 50 meters - connect. 50-100 meters - only if it is not cross nay other line

In [10]:
# Extend
new_network2 =gpd.read_file(f'{path_network}/network_ra.shp')
extend_lines_f= extend_lines(new_network2,100)

extend_lines_f.to_file(f'{cur_path}/extend_lines.shp')

In [11]:
def find_intersection_points2(row):
    r"""
    find the intersection points between the two lines
    :param row:
    :return:
    """
    try:
        line_1 = extend_lines_f.loc[row['index']]
        line_2 = extend_lines_f.loc[row['index_right']]
        pnt = line_1.geometry.intersection(line_2.geometry)
        # If there are more than one intersection between two lines, one of the lines should be deleted.
        if isinstance(pnt,MultiPoint):
            temp_line= line_1.name if line_1.length< line_2.length else line_2.name
            if temp_line not in lines_to_delete:
                lines_to_delete.append(temp_line)
            return
        # If it is first or end continue OR if there is no intersection between the two lines
        if len(pnt.coords)==0 or pnt.coords[0]==line_1.geometry.coords[0] or pnt.coords[0]==line_1.geometry.coords[-1]:
            return
        inter_pnt_dic['geometry'].append(pnt)
        inter_pnt_dic['name'].append(row['index'])
    except:
        print(f"{row['index']},{row['index_right']}:{pnt}")

In [37]:
s_join_0

,index,geometry,index_right,str_name,highway,bearing,length,is_simplif,group
0,0,"POLYGON ((855892.982 5634190.602, 855893.013 5...",3851,None,None,NaN,169.671342,NaN,NaN
1,3851,"POLYGON ((855971.235 5634074.050, 855979.628 5...",3851,None,None,NaN,169.671342,NaN,NaN
2,3860,"POLYGON ((856062.887 5633908.136, 856125.848 5...",3851,None,None,NaN,169.671342,NaN,NaN
3,6860,"POLYGON ((856155.187 5633991.352, 856155.182 5...",3851,None,None,NaN,169.671342,NaN,NaN
4,6900,"POLYGON ((856042.930 5634124.152, 856043.775 5...",3851,None,None,NaN,169.671342,NaN,NaN
...,...,...,...,...,...,...,...,...,...
54753,9489,"POLYGON ((850058.502 5637761.047, 850058.540 5...",9488,Viale dei Mughetti,tertiary,109.473333,137.972382,1.0,0.0
54754,9476,"POLYGON ((849982.873 5637753.108, 849982.570 5...",9477,Via delle Pervinche,residential,10.107143,177.377032,1.0,1.0
54755,9477,"POLYGON ((850044.319 5637952.719, 850044.622 5...",9477,Via delle Pervinche,residential,10.107143,177.377032,1.0,1.0
54756,9488,"POLYGON ((849966.391 5637791.094, 849966.005 5...",9477,Via delle Pervinche,residential,10.107143,177.377032,1.0,1.0


In [12]:
# Create buffer around each element
buffer_around_lines= extend_lines_f['geometry'].buffer(cap_style=3, distance=1, join_style=3)
buffer_around_lines.to_file(f'{cur_path}/buffer2.shp')

# s_join between buffer to lines
s_join_0 =gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_lines,crs=project_crs),right_df=extend_lines_f).reset_index()

# delete lines belong to the buffer
s_join = s_join_0[s_join_0['index']!=s_join_0['index_right']]
s_join.to_file(f'{cur_path}/s_join2.shp')

# Find new intersections that are not at the beginning or end of the line
lines_to_delete =[]
inter_pnt_dic = {'geometry':[],'name':[]}
s_join.apply(find_intersection_points2, axis=1)
inter_pnt_gdf = GeoDataFrame(inter_pnt_dic,crs=project_crs)
inter_pnt_gdf.to_file(f'{cur_path}/inter_pnt2.shp')



In [13]:
# Split string line by points
segments = {'geometry':[],'org_id':[]}
# Groupby points name (which is the line they should split)
for group_pnts in inter_pnt_gdf.groupby('name'):
    points  = group_pnts[1]
    points['is_split'] = True

    # get the line to split by comparing the name
    row = extend_lines_f.loc[group_pnts[0]]
    current = list(row.geometry.coords)
    points_line = [Point(x) for x in current]
    points_line_gdf = GeoDataFrame(geometry=points_line,crs=project_crs)
    points_line_gdf['is_split'] = False

    # append all the points together (line points and split points)
    line_all_pnts = points_line_gdf.append(points)

    # Find the distance of each point form the begining of the line on the line.
    line_all_pnts['dis_from_the_start'] = line_all_pnts['geometry'].apply(lambda x:row.geometry.project(x))
    line_all_pnts.sort_values('dis_from_the_start',inplace=True)

    # split the line
    seg =[]
    for point in line_all_pnts.iterrows():
        prop = point[1]
        seg.append(prop['geometry'])
        if prop['is_split']:
            segments['geometry'].append(LineString(seg))
            segments['org_id'].append(row.name)
            seg = [prop['geometry']]
    segments['geometry'].append(LineString(seg))
    segments['org_id'].append(row.name)
network_split = GeoDataFrame(data=segments,crs=project_crs)
cols_no_geometry = extend_lines_f.columns[:-1]
network_split_final = network_split.set_index('org_id')
network_split_final[cols_no_geometry] =extend_lines_f[cols_no_geometry]
network_split_final.to_file(f'{cur_path}/only_split.shp')
# remove old and redundant line from our network and update with new one
network_split = extend_lines_f.drop(index=network_split_final.index.unique()).append(network_split_final).drop(index= lines_to_delete)

In [14]:
network_split['length'] = network_split.length

In [22]:
pnt_dic = {}
first_last_dic = {'geometry': [], 'line_name': [], 'position': []}
network_split.apply(send_pnts, axis=1)
dangel_line = [item[0] for item in pnt_dic.values() if len(item) == 1]

# save all the deadend_list points
deadend_pnts_list = [item[0] for item in pnt_dic.items() if len(item[1]) == 1]

In [23]:
not_connected_lines = [j for p, j in enumerate(dangel_line) if dangel_line[:p].count(j) == 1]
only_connected_gdf = network_split.drop(index= not_connected_lines)


In [41]:
deadend_list = [j for v, j in enumerate(dangel_line) if dangel_line[:v].count(j) == 0]
only_deadend = [item for item in deadend_list  if item not in not_connected_lines]

In [45]:
connected_gdf = only_connected_gdf.loc[only_deadend]

In [48]:
delete_short =connected_gdf[connected_gdf.length < 30]
delete_short

,str_name,highway,bearing,length,is_simplif,group,geometry
7365,Via Moncrivello,residential,148.4,1.164153e-10,0.0,NaN,"LINESTRING (858009.888 5635710.978, 858009.888..."
7367,Via Eusebio Bava,residential,221.8,1.164153e-10,0.0,NaN,"LINESTRING (857544.788 5632598.408, 857544.788..."
7377,Viale dei Partigiani,tertiary,69.2,4.656613e-10,0.0,NaN,"LINESTRING (856053.585 5632867.735, 856053.585..."
7391,Via Sant'Anselmo,residential,16.6,1.164153e-10,0.0,NaN,"LINESTRING (854926.525 5630385.842, 854926.525..."
7395,Via Camillo Benso Conte di Cavour,residential,115.9,1.164153e-10,0.0,NaN,"LINESTRING (856059.631 5631391.756, 856059.631..."
...,...,...,...,...,...,...,...
9777,None,None,NaN,2.328306e-10,NaN,NaN,"LINESTRING (858884.620 5636319.984, 858884.620..."
9779,None,None,NaN,1.304782e+01,NaN,NaN,"LINESTRING (859143.298 5633403.582, 859151.930..."
9789,None,None,NaN,1.164153e-10,NaN,NaN,"LINESTRING (859662.215 5637120.473, 859662.215..."
9803,None,None,NaN,1.164153e-10,NaN,NaN,"LINESTRING (861250.085 5632519.354, 861250.085..."


In [52]:
final = only_connected_gdf.drop(index= delete_short.index)
final

,str_name,highway,bearing,length,is_simplif,group,geometry
1,Via Don Giovanni Bosco,residential,112.4,85.627516,0.0,NaN,"LINESTRING (852685.815 5634980.859, 852764.985..."
2,Via Don Giovanni Bosco,residential,93.0,189.806264,0.0,NaN,"LINESTRING (852496.271 5634990.839, 852522.175..."
3,Via Alfonso Bonafous,residential,209.8,168.972995,0.0,NaN,"LINESTRING (856590.513 5631546.140, 856584.658..."
4,Viale Giacomo Curreno,tertiary,107.8,15.787604,0.0,NaN,"LINESTRING (856683.420 5630199.603, 856698.448..."
5,Via Vittorio Amedeo Gioanetti,residential,355.3,152.543606,0.0,NaN,"LINESTRING (857039.119 5630823.110, 857048.938..."
...,...,...,...,...,...,...,...
9792,None,None,NaN,44.958939,NaN,NaN,"LINESTRING (859433.402 5632316.637, 859425.998..."
9796,None,None,NaN,227.167126,NaN,NaN,"LINESTRING (860185.722 5633386.276, 860212.983..."
9796,None,None,NaN,35.718850,NaN,NaN,"LINESTRING (860412.062 5633398.254, 860425.458..."
9801,None,None,NaN,935.420676,NaN,NaN,"LINESTRING (861298.250 5631401.281, 861242.646..."


In [51]:
delete_short

,str_name,highway,bearing,length,is_simplif,group,geometry
7365,Via Moncrivello,residential,148.4,1.164153e-10,0.0,NaN,"LINESTRING (858009.888 5635710.978, 858009.888..."
7367,Via Eusebio Bava,residential,221.8,1.164153e-10,0.0,NaN,"LINESTRING (857544.788 5632598.408, 857544.788..."
7377,Viale dei Partigiani,tertiary,69.2,4.656613e-10,0.0,NaN,"LINESTRING (856053.585 5632867.735, 856053.585..."
7391,Via Sant'Anselmo,residential,16.6,1.164153e-10,0.0,NaN,"LINESTRING (854926.525 5630385.842, 854926.525..."
7395,Via Camillo Benso Conte di Cavour,residential,115.9,1.164153e-10,0.0,NaN,"LINESTRING (856059.631 5631391.756, 856059.631..."
...,...,...,...,...,...,...,...
9777,None,None,NaN,2.328306e-10,NaN,NaN,"LINESTRING (858884.620 5636319.984, 858884.620..."
9779,None,None,NaN,1.304782e+01,NaN,NaN,"LINESTRING (859143.298 5633403.582, 859151.930..."
9789,None,None,NaN,1.164153e-10,NaN,NaN,"LINESTRING (859662.215 5637120.473, 859662.215..."
9803,None,None,NaN,1.164153e-10,NaN,NaN,"LINESTRING (861250.085 5632519.354, 861250.085..."


In [50]:
only_connected_gdf

,str_name,highway,bearing,length,is_simplif,group,geometry
1,Via Don Giovanni Bosco,residential,112.4,8.562752e+01,0.0,NaN,"LINESTRING (852685.815 5634980.859, 852764.985..."
2,Via Don Giovanni Bosco,residential,93.0,1.898063e+02,0.0,NaN,"LINESTRING (852496.271 5634990.839, 852522.175..."
3,Via Alfonso Bonafous,residential,209.8,1.689730e+02,0.0,NaN,"LINESTRING (856590.513 5631546.140, 856584.658..."
4,Viale Giacomo Curreno,tertiary,107.8,1.578760e+01,0.0,NaN,"LINESTRING (856683.420 5630199.603, 856698.448..."
5,Via Vittorio Amedeo Gioanetti,residential,355.3,1.525436e+02,0.0,NaN,"LINESTRING (857039.119 5630823.110, 857048.938..."
...,...,...,...,...,...,...,...
9801,None,None,NaN,1.219680e+02,NaN,NaN,"LINESTRING (860653.710 5632048.827, 860536.132..."
9803,None,None,NaN,1.501916e+02,NaN,NaN,"LINESTRING (861131.119 5632427.677, 861234.929..."
9803,None,None,NaN,1.164153e-10,NaN,NaN,"LINESTRING (861250.085 5632519.354, 861250.085..."
9804,None,None,NaN,1.243722e+02,NaN,NaN,"LINESTRING (860825.484 5635404.316, 860775.113..."


In [53]:
final.to_file(f'{cur_path}/final.shp')